***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-002e  GCE       3                                             RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import math
import builtins as pybuiltins


import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Feb 29 08:23 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'noam209263805' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and not b.name.startswith("postings_gcp"):
        paths.append(full_path+b.name)

In [8]:
parquetFile = spark.read.parquet(*paths)
doc_title_pairs = parquetFile.select("title", "id").rdd
# doc_anchor_pairs = parquetFile.select("anchor_text", "id").rdd
# doc_body_pairs = parquetFile.select("text", "id").rdd

In [9]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [10]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [11]:
from inverted_index_gcp import InvertedIndex

**Project**


In [12]:
# stop words
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)


# tokenize and stemming
def tokenize_stemming(text):
    list_token = []
    stemmer = PorterStemmer()
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    for token in tokens:
        if token not in english_stopwords:
            list_token.append(stemmer.stem(token))
    return list_token


In [29]:
# tf
# normalizion by max frequency term
def term_frequency(text, id):
#     tokens_stem = tokenize_stemming(text)
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]

    # Count the frequency of each word that is not a stopword
    word_freq = {}
    doc_size = 0
    word_freq["doc_lenght"] = doc_size

    for token in tokens:
        doc_size += 1
        word_freq["doc_lenght"] = doc_size
        if token not in all_stopwords:
            if token not in word_freq:
                word_freq[token] = 1
            else:
                word_freq[token] += 1
    # freq after normalizied
    lst_tuples = [(token, (id, freq, doc_size)) for token, freq in word_freq.items()]
    return lst_tuples


def reduce_word_counts(unsorted_pl):
  sorted_pl = sorted(unsorted_pl, key=lambda x: x[0])
  return sorted_pl

# df
def calculate_df(postings):
  rdd_token_doc_freq = postings.map(lambda x: (x[0], len(x[1]))).reduceByKey(lambda a, b: a + b)
  return rdd_token_doc_freq


# doc len
# def doc_lenght(text, doc_id):
#   tokens = tokenize_stemming(text)
#   return((doc_id,len(tokens)))


In [30]:


NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings, index_name):
  def map_to_buckets(pair):
        word, posting_list = pair
        bucket_id = token2bucket_id(word)
        return bucket_id, [(word, posting_list)]

  def reduce_to_disk(bucket, word_posting_lists):
    # Write posting lists for the bucket to disk
    # posting_locs = InvertedIndex.write_a_posting_list((bucket, word_posting_lists), 'postings_gcp', bucket_name)
    return posting_locs

  # # Partition the postings into buckets
  # buckets = postings.map(map_to_buckets)

  # # Group postings by bucket ID
  # grouped_buckets = buckets.reduceByKey(lambda x, y: x + y)

  # # Write each bucket to disk and get posting locations
  # bucket_posting_locs = grouped_buckets.map(lambda x: reduce_to_disk(*x))
  # return bucket_posting_locs

  # ---------------------------------new ---------------------------
  tempRDD = postings.map(lambda x:(token2bucket_id(x[0], NUM_BUCKETS),[(x[0],x[1])]))
  ldRDD = tempRDD.reduceByKey(lambda a,b: a+b)
  posting_list =ldRDD.map(lambda x: InvertedIndex.write_a_posting_list(x, ".", f'bucket_{index_name}'))
  return posting_list
  # -----------------------------------------------------------------


def create_posting_locs(index_name, postings_filtered):
  posting_locs_list = partition_postings_and_write(postings_filtered, index_name).collect()

  super_posting_locs = defaultdict(list)
  for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)
  return super_posting_locs

In [31]:
def create_index(index_name, doc_pairs):

    word_counts = doc_pairs.flatMap(lambda x: term_frequency(x[0], x[1]))
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    postings_filtered = postings.filter(lambda x: len(x[1])>50)

#     dl_rdd = doc_pairs.map(lambda x: doc_lenght(x[0], x[1]))
#     dl = dl_rdd.collectAsMap()
    w2df = calculate_df(postings_filtered)
    w2df_dict = w2df.collectAsMap()

    # Create inverted index instance
    inverted = InvertedIndex()
    inverted.df = w2df_dict
    inverted.N = doc_pairs.count()
#     inverted.doc_len = dl
    inverted.posting_locs = create_posting_locs(index_name, postings_filtered)

    inverted.write_index('.', f'index_{index_name}')

    index_src = f"index_{index_name}.pkl"
    index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
    !gsutil cp $index_src $index_dst
    
#     return inverted

In [32]:
TUPLE_SIZE = 6
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing


def read_posting_list(inverted, w):
  with closing(MultiFileReader(".", bucket_name)) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [33]:
# weight = tf*idf
def calculate_tf_idf(index, term, tf, doc_id):
  tf_idf = tf* (math.log2(index.N/index.df[term]))
  return tf_idf


def cosine_similarity(query, index):
    """ Returns: {doc_id:cosine score} """

    dict_cosine_sim = {}
    doc_weights_dict = {}
    query_dict = dict(term_frequency(query ,0))

    # create lists of keys and values from the query_dict
    query_list_keys = list(query_dict.keys())
    query_list_values = list(query_dict.values())

    # create list of items from index.df dictionery keys
    index_df_list_keys = list(index.df.keys())


    for term in query_list_keys:
      w_term_query = (query_dict[term][1]/len(query))* math.log2(index.N/index.df[term])

      if term in index_df_list_keys:
        posting_list = index.read_a_posting_list(".", term, bucket_name)
        
        for doc_id, freq, doc_len in posting_list:  
          w_term_doc = calculate_tf_idf(index, term, freq/doc_len, doc_id)
          if doc_id in dict_cosine_sim.keys():
            dict_cosine_sim[doc_id] += (w_term_doc)*(w_term_query)
            doc_weights_dict[doc_id] += (w_term_doc) ** 2
          else:
            dict_cosine_sim[doc_id] = (w_term_doc)*(w_term_query)
            doc_weights_dict[doc_id] = (w_term_doc) ** 2


    for doc_id in list(dict_cosine_sim.keys()):
      Word_doc_id__weight = doc_weights_dict[doc_id];
      dict_cosine_sim[doc_id] /= ( math.sqrt(pybuiltins.sum(value[1] ** 2 for value in query_list_values) * Word_doc_id__weight ))



    sorted_docs = sorted(dict_cosine_sim.items(), key=lambda x: x[1], reverse=True)

    print(sorted_docs)
    return dict_cosine_sim


In [34]:
# def BM25(query, index, k, b, avg_doclen):
#     bm_score = {}
#     # create lists of keys and values from the query_dict
#     query_list_keys = list(query_dict.keys())
#     query_list_values = list(query_dict.values())
#     # create list of items from index.df dictionery keys
#     index_df_list_keys = list(index.df.keys())
    
#     idf_dict = calc_idf(query, N, index)
    
#     for term in query_list_keys:
#         if term in index_df_list_keys:
#             posting_list = index.read_a_posting_list(".", term, bucket_name)
#             for doc_id, freq in posting_list:
#                 B = 1 - b + b * (index.dl[doc_id] / avg_doclen)
#                 idf = idf_dict[term]
#                 tf = freq / index.dl[doc_id]
#                 if doc_id in bm_score:
#                     bm_score[doc_id] += (idf * freq * (k + 1)) / ((freq + B * k))
#                 else:
#                     bm_score[doc_id] = (idf * freq * (k + 1)) / ((freq + B * k))
    
#     return dict(sorted(bm_score.items(), key=lambda x: x[1], reverse=True))
            

In [35]:
import time
def search_by_title(inverted, query):
    """ Returns: n best docs """
    start_time = time.time()  # Record the start time

    query_filterd = tokenize_stemming(query)
    cosSim_score = cosine_similarity(query, inverted)

    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    print("Function execution time:", execution_time, "seconds")
    return cosSim_score

In [36]:
create_index("title", doc_title_pairs)
# idx = InvertedIndex.read_index("postings_gcp", 'index_title', bucket_name)


Copying file://index_title.pkl [Content-Type=application/octet-stream]...
/ [1 files][957.9 KiB/957.9 KiB]                                                
Operation completed over 1 objects/957.9 KiB.                                    


In [39]:
idx = InvertedIndex.read_index("postings_gcp", 'index_title', bucket_name)
idx.read_a_posting_list(".", "genetic", bucket_name)

[(12383, 65538, 2),
 (12385, 65538, 2),
 (12424, 65538, 2),
 (12437, 65538, 2),
 (12942, 65537, 1),
 (40254, 65538, 2),
 (44353, 65539, 3),
 (59013, 65539, 3),
 (72016, 65538, 2),
 (144854, 65545, 9),
 (157941, 65538, 2),
 (158005, 65538, 2),
 (251487, 65538, 2),
 (274192, 65538, 2),
 (310782, 65538, 2),
 (340952, 65538, 2),
 (347560, 65539, 3),
 (364423, 65538, 2),
 (371709, 65538, 2),
 (403627, 65538, 2),
 (550948, 65539, 3),
 (554546, 65539, 3),
 (555213, 65539, 3),
 (555480, 65539, 3),
 (562180, 65539, 3),
 (887836, 65539, 3),
 (901149, 65539, 3),
 (1090609, 65538, 2),
 (1105886, 65539, 3),
 (1217294, 65539, 3),
 (1325445, 65539, 3),
 (1431111, 65538, 2),
 (1505166, 65538, 2),
 (1652585, 65538, 2),
 (1669966, 65538, 2),
 (1693323, 65538, 2),
 (1795026, 65538, 2),
 (1840306, 65539, 3),
 (1988101, 65538, 2),
 (2072213, 65541, 5),
 (2162538, 65538, 2),
 (2203410, 65539, 3),
 (2228439, 65538, 2),
 (2520461, 65538, 2),
 (2722642, 65538, 2),
 (2792293, 65538, 2),
 (2854670, 65540, 4),
 (

In [ ]:
# query = "genetic"
# res = search_by_title(idx, query)

In [ ]:
# create_index("body", doc_body_pairs)
# idx = InvertedIndex.read_index("postings_gcp", 'index_body', bucket_name)